##Install Bert as a Service

In [ ]:
!pip install bert-serving-server  # server
!pip install bert-serving-client  # client, independent of `bert-serving-server`

##Configuration variables

In [ ]:
root = '/content/drive/My Drive/nlp-lab'

#BERT Models
#BERT_model_location = root + '/models/BERT-models/BERT-base-uncased/uncased_L-12_H-768_A-12'
BERT_model_location = root + '/models/BERT-models/BERT-base-multilingual-cased/multi_cased_L-12_H-768_A-12'

#Data location
processed_data_location = root + '/data/processed-data/organic'
#processed_data_location = root + '/data/processed-data/organic'


#exported_file_name = 'processed_data_with_embeddings_BERT-base-uncased.json'
exported_file_name = 'processed_data_with_embeddings_BERT-base-miltilingual-cased.json'

##Imports

In [ ]:
import pandas as pd
from bert_serving.client import BertClient
from google.colab import drive
from sys import path

##Mount Drive

In [ ]:
drive.mount('/content/drive', force_remount=True)
path.append(root)

Mounted at /content/drive


##Downgrade Tensorflow version

In [ ]:
%tensorflow_version 1.x
import tensorflow
print(tensorflow.__version__)

TensorFlow 1.x selected.
1.15.2


#Start Server



In [ ]:
#Warning: the parameter -cased_tokenization should only be used for cased models
get_ipython().system_raw('bert-serving-start -model_dir "$BERT_model_location" -num_worker=4 -max_seq_len=100 -cased_tokenization | tee /tmp/output2.txt &')
#get_ipython().system_raw('bert-serving-start -model_dir "$BERT_model_location" -num_worker=4 -max_seq_len=100 | tee /tmp/output2.txt &')
#!bert-serving-start -model_dir "$BERT_model_location" -num_worker=4 -max_seq_len=100 | tee /tmp/output2.txt

##Load Dataset

In [ ]:
data_df = pd.read_json(processed_data_location + '/processed_data_without_embeddings_no_outliers.json')

##Test if BERT server is working

In [ ]:
#bc = BertClient(timeout = 100000)
bc = BertClient()
print(bc.server_status)

##Get embeddings for our sentences

In [ ]:
sentences_encoded = bc.encode(data_df['sentence_text'].tolist())

##Add embeddings to our dataframe

In [ ]:
data_with_embeddings = pd.concat([data_df.reset_index(drop=True),
                                  pd.DataFrame(sentences_encoded)], axis=1)

##Export results

In [ ]:
data_with_embeddings.to_json(processed_data_location + '/' + exported_file_name)